In [775]:
from curses import flash
import os
from pickle import TRUE
from re import L
import requests
import validators
import gzip
import shutil
from urllib.parse import urlparse
import random

import sys
import math
import xarray as xa
import numpy as np
from rio_cogeo import cog_validate
import rioxarray

# Mapping
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [801]:
def open_file(input_path):
    return xa.open_dataset(input_file_path, engine="netcdf4", decode_coords='all', decode_times=False)

def copy_lat_lon_data(file, var_name1, var_name2, var_name3):
    lat = file[var_name1].data.copy()
#     lat = lat[::-1]
    lon = file[var_name2].data.copy()
    data = file[var_name3].data[0].copy()
    return lat, lon, data

def make_new_xarray(lat, lon, data):
#     file = xa.Dataset(
#         data_vars={"flash_extent": (["longitude", "latitude"], data)},
#         coords={["longitude"]: lon, ["latitude"]: lat},
#         attrs={"instrument_ID": "NALMA"}
#     )
    file = xr.Dataset(
        {
            "flash_extent": (["longitude", "latitude"], data),
        },
        coords={
            "longitude": (["longitude"], lon),
            "latitude": (["latitude"], lat),
        },
    )
    return file

def generate_cog(file, variable_name, latitude, longitude):
    file = file[variable_name]
#     file = file[::-1]
    file = file.transpose(latitude, longitude)
    file.rio.set_spatial_dims(x_dim=longitude, y_dim=latitude, inplace=True)
    file.rio.crs
    file.rio.set_crs('epsg:4326')
    name=f"S2A_20160724_135032_27XVB_B{random.randint(0,9)}{random.randint(0,9)}{random.randint(0,9)}.tif"
    cog_path=f"/home/asubedi/test_cog/cogs/{name}"
    print(name)
    file.rio.to_raster(rf'{cog_path}', driver='COG')

In [802]:
def nalma():
    input_file_path = "/home/asubedi/test_cog/NALMA_20230629_235000_600_10src_0.0109deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def wtlma():
    input_file_path = "/home/asubedi/test_cog/WTLMA_20170531_235000_600_10src_0.0108deg-dx_flash_extent.nc"
    variable_name = "flash_extent"
    lat='latitude'
    lon='longitude'
    return input_file_path, variable_name, lat, lon

def trmm_lis():
    input_file_path = '/home/asubedi/Desktop/data/raw-files/TRMM-LIS/lis_vhrfc_1998_2013_v01.nc'
    variable_name = "VHRFC_LIS_FRD"
    lat="Latitude"
    lon="Longitude"
    return input_file_path, variable_name, lat, lon

In [803]:
input_file_path, variable_name, latitude, longitude = nalma()
file = open_file(input_file_path)
lat, lon, data = copy_lat_lon_data(file, latitude, longitude, variable_name)
file = make_new_xarray(lat, lon ,data)
# generate_cog(file, variable_name, latitude, longitude)

In [804]:
index = 0
for arr in data:
    if np.any(arr != 0) == True:
        print(f"Index: {index}, lon:{lon[index]}")
    index += 1

print("--------------------------------------")
    
index = 0
for column in data.T:
    if np.any(column != 0) == True:
        print(f"Index: {index}, lat: {lat[index]}")
    index += 1

Index: 154, lon:-87.13673400878906
Index: 155, lon:-87.12581634521484
Index: 156, lon:-87.11489868164062
Index: 157, lon:-87.1039810180664
Index: 158, lon:-87.09306335449219
Index: 159, lon:-87.08214569091797
--------------------------------------
Index: 79, lat: 33.633785247802734
Index: 80, lat: 33.642799377441406
Index: 81, lat: 33.65181350708008
Index: 82, lat: 33.66082763671875
Index: 83, lat: 33.66984176635742
Index: 84, lat: 33.678855895996094
Index: 85, lat: 33.687870025634766
Index: 86, lat: 33.69688415527344
Index: 88, lat: 33.71491241455078
Index: 89, lat: 33.72392654418945
Index: 90, lat: 33.732940673828125


In [805]:
def delete_row_col(lat, lon, data):
    #finding row with all zeroes    
    zero_row = np.where(np.all(data == 0, axis=1))[0]
    #removing lon with all zeroes
    lon = np.delete(lon, zero_row)
    # Find rows with non-zero elements
    non_zero_rows = np.any(data != 0, axis=1)
    # Filter the array based on non-zero rows
    data = data[non_zero_rows]
    
    #finding columns with all zeroes
    zero_columns = np.where(np.all(data == 0, axis=0))[0]
    #removing lat with all zeroes
    lat = np.delete(lat, zero_columns)
    lat = lat[::-1]
    # Find rows with non-zero elements
    non_zero_rows = np.any(data != 0, axis=1)
    # Find columns with non-zero elements
    non_zero_columns = np.any(data != 0, axis=0)
    # Filter the array based on non-zero rows and columns
    data = data[non_zero_rows][:, non_zero_columns]
    return lat, lon, data

In [806]:
lat, lon, data = delete_row_col(lat, lon, data)

In [807]:
print(len(lat), len(lon), data.shape)

11 6 (6, 11)


In [808]:
file = make_new_xarray(lat, lon, data)

In [811]:
file.to_netcdf("/mnt/c/Users/asubedi/Desktop/testing_nalma_new_array.nc")

In [810]:
generate_cog(file, variable_name, latitude, longitude)

S2A_20160724_135032_27XVB_B817.tif


ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_name: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
ERROR 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /home/asubedi/miniconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
